Arrange the sales of each store per category for Corporacion Favorita.

Save all the store sales per category in a pickle file so we can easily delve into store level when needed.

Updates:

10.10.2020 - Adapt the script to CFav data.

In [4]:
import sys
sys.path

['/Users/carlos.aguilar/Documents/carlosDev/causalCannibalisation/prepare_datasets',
 '/usr/local/Cellar/python@3.8/3.8.7/Frameworks/Python.framework/Versions/3.8/lib/python38.zip',
 '/usr/local/Cellar/python@3.8/3.8.7/Frameworks/Python.framework/Versions/3.8/lib/python3.8',
 '/usr/local/Cellar/python@3.8/3.8.7/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/carlos.aguilar/Library/Python/3.8/lib/python/site-packages',
 '/usr/local/lib/python3.8/site-packages',
 '/usr/local/lib/python3.8/site-packages/psycopg2-2.8.5-py3.8-macosx-10.15-x86_64.egg',
 '/usr/local/lib/python3.8/site-packages/causalnex-0.7.0-py3.8.egg',
 '/usr/local/lib/python3.8/site-packages/wrapt-1.11.2-py3.8-macosx-10.15-x86_64.egg',
 '/usr/local/lib/python3.8/site-packages/sagemaker_xgboost_container-2.0-py3.8.egg',
 '/usr/local/lib/python3.8/site-packages/smdebug-0.4.13-py3.8.egg',
 '/usr/local/lib/python3.8/site-packages/sagemaker_containers-2.8.6.post2-py3.8-macosx-10.15-x86_64.egg'

In [1]:
!pwd

/Users/carlos.aguilar/Documents/carlosDev/causalCannibalisation/prepare_datasets


In [2]:
from utils import fcn_helpers as fhelp

ImportError: cannot import name 'fcn_helpers' from 'utils' (/usr/local/lib/python3.8/site-packages/EF_EdTech_Data_Science_utilities-0.3-py3.8.egg/utils/__init__.py)

In [3]:
from ..utils.fcn_helpers import fcn_helpers as fhelp

ImportError: attempted relative import with no known parent package

In [4]:
import utils.fcn_helpers as fhelp
import utils.fcn_simulation as fsim

ModuleNotFoundError: No module named 'utils.fcn_helpers'

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils.fcn_helpers as fhelp
import utils.fcn_simulation as fsim
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
#plt.rcParams['figure.figsize'] = [14, 7]
fhelp.makeFolder('results')
pd.options.display.max_rows = None
# Use v2.0 colour cycle
def_colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
# Fig sizes
fig_h = 10
fig_w = 18

In [9]:
outputBaseFolder = '~/Google Drive/order/Machine Learning Part/data/CorporacionFavorita'
min_date = '2017-01-01'
max_date = '2017-08-15'

### Prepare the CF data here in a similar format than Walmart

In [10]:
cf_dataFolder = '/Users/carlos.aguilar/Documents/Kaggle/Grocery Sales Forecasting/data'
cf_csvFolder = os.path.join(cf_dataFolder, 'csv')
# Read the stores
csvFile = 'stores.csv'
csvPath = os.path.join(cf_csvFolder, csvFile)
df_stores = pd.read_csv(csvPath, delimiter=',')
df_stores.rename(columns={'state':'state_id'}, inplace=True)

#df_stores['store_id'] = df_stores[['state_id', 'store_nbr']].apply(lambda st: f'{st[0]}_{st[1]}', axis=1)
# Include the type and cluster in the store name
df_stores['store_id'] = df_stores[['state_id', 'store_nbr', 'type', 'cluster']].apply(lambda st: f'{st[0]}_{st[1]}_{st[2]}_{st[3]}', axis=1)


df_stores['store_id'] = df_stores['store_id'].apply(lambda string: string.replace(' ', '_'))
mapper_store_id = dict(zip(df_stores['store_nbr'],df_stores['store_id']))

#fhelp.to_random_excel_file(df_stores)
df_stores.head(12)

,store_nbr,city,state_id,type,cluster,store_id
0,1,Quito,Pichincha,D,13,Pichincha_1_D_13
1,2,Quito,Pichincha,D,13,Pichincha_2_D_13
2,3,Quito,Pichincha,D,8,Pichincha_3_D_8
3,4,Quito,Pichincha,D,9,Pichincha_4_D_9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,Santo_Domingo_de_los_Tsachilas_5_D_4
5,6,Quito,Pichincha,D,13,Pichincha_6_D_13
6,7,Quito,Pichincha,D,8,Pichincha_7_D_8
7,8,Quito,Pichincha,D,8,Pichincha_8_D_8
8,9,Quito,Pichincha,B,6,Pichincha_9_B_6
9,10,Quito,Pichincha,C,15,Pichincha_10_C_15


In [11]:
mapper_family_to_category = fhelp.cfav_get_map_dept_to_cat()

In [12]:
# Read the items
csvFile = 'items.csv'
csvPath = os.path.join(cf_csvFolder, csvFile)
df_items = pd.read_csv(csvPath, delimiter=',')
df_items.head(2)

# let's make some changes to have the Walmart structure
# family > dept_id
this_var = 'family'
families = df_items[this_var].unique().tolist()
dept_ids = [*map(lambda str: str.replace(' ', '_').replace('/','_'), families)]
mapper_to_dept_id = dict(zip(families, dept_ids))
df_items['dept_id'] = df_items[this_var].apply(lambda str: mapper_to_dept_id[str])
df_items['cat_id'] = df_items[this_var].apply(lambda x: mapper_family_to_category[x])

# class > category
#this_var = 'class'
#sku_classes = df_items[this_var].unique().tolist()
#categories = [*map(lambda x: f'cat_{x}', sku_classes)]
#mapper_to_category = dict(zip(sku_classes, categories))
#df_items['cat_id'] = df_items[this_var].apply(lambda x: mapper_dep_to_category[x])

df_items.rename(columns={'item_nbr':'item_id'}, inplace=True)

mapper_dept_to_category = dict(zip(df_items['dept_id'], df_items['cat_id']))

df_items.sort_values(by=['cat_id', 'dept_id'],ascending=True, inplace=True)
fhelp.to_random_excel_file(df_items)

# Save to a file
foldername = os.path.join(os.path.expanduser(outputBaseFolder), 'sku_hierarchy')
fhelp.makeFolder(foldername)
filename = os.path.join(foldername, 'items.pickle')
fhelp.toPickleFile(df_items, filename)
        

df_items.head()

Pickle file saved to /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/data/CorporacionFavorita/sku_hierarchy/items.pickle


,item_id,family,class,perishable,dept_id,cat_id
53,121964,AUTOMOTIVE,6810,0,AUTOMOTIVE,AUTOMOTIVE
120,172995,AUTOMOTIVE,6824,0,AUTOMOTIVE,AUTOMOTIVE
256,270522,AUTOMOTIVE,6806,0,AUTOMOTIVE,AUTOMOTIVE
298,312317,AUTOMOTIVE,6810,0,AUTOMOTIVE,AUTOMOTIVE
529,461432,AUTOMOTIVE,6810,0,AUTOMOTIVE,AUTOMOTIVE


### Read the holiday-events

In [17]:
# Read the holiay
csvFile = 'holidays_events.csv'
csvPath = os.path.join(cf_csvFolder, csvFile)
df_events = pd.read_csv(csvPath, delimiter=',', parse_dates=['date'])

idx_A = df_events.date > min_date
idx_B = df_events.date <= max_date

df_events = df_events[idx_A & idx_B].copy()
df_events.head(4)


,date,type,locale,locale_name,description,transferred
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
299,2017-02-27,Holiday,National,Ecuador,Carnaval,False
300,2017-02-28,Holiday,National,Ecuador,Carnaval,False
301,2017-03-02,Holiday,Local,Manta,Fundacion de Manta,False


In [20]:
# Read the pickle file with the data from 2017
pickleFile = os.path.join(cf_dataFolder, 'pickle', 'training_set_2017.pickle')
df_train = fhelp.readPickleFile(pickleFile)
df_train.rename(columns={'item_nbr':'item_id', 'unit_sales': 'sales'}, inplace=True)
df_train.head(2)

,date,store_nbr,item_id,sales,onpromotion
35229871,2017-01-01,25,99197,1.0,False
35229872,2017-01-01,25,103665,7.0,False


## Iterate through all the departments and stores

In [22]:
all_dept_id = df_items['dept_id'].unique().tolist()

# Find the sales for the current dept_id
idx_dept_id = df_items.dept_id==dept_id
list_skus = df_items[idx_dept_id].item_id.unique().tolist()

df_sales_dept_id = df_train[df_train.item_id.isin(list_skus)].copy()
df_sales_dept_id.head(2)


In [23]:
#dept_id = 'BREAD_BAKERY' # comes from dept_ids
all_dept_id = df_items['dept_id'].unique().tolist()
#all_dept_id.remove('GROCERY_I')

idx_dates = pd.date_range(min_date, max_date)


for dept_id in all_dept_id:
    
    # Find the sales for the current dept_id
    idx_dept_id = df_items.dept_id==dept_id
    list_skus = df_items[idx_dept_id].item_id.unique().tolist()
    df_sales_dept_id = df_train[df_train.item_id.isin(list_skus)].copy()

    cat_id = mapper_dept_to_category[dept_id]
    
    list_store_id = df_sales_dept_id.store_nbr.unique().tolist()
    total_stores = len(list_store_id)

    for idx_this_store, current_store_id in enumerate(list_store_id):

        print(f'{idx_this_store}/{total_stores}-Working on {current_store_id}...')

        df_sales_dept_id_store_id = df_sales_dept_id[df_sales_dept_id.store_nbr==current_store_id]
        total_units = np.zeros(shape=idx_dates.shape[0])
        df_store_sales = pd.DataFrame({'date':idx_dates, 'weekdays':idx_dates.day_name()})

        store_name = mapper_store_id[current_store_id]
        num_skus = len(list_skus)

        print(f'Number of SKUs {num_skus}')

        for current_sku in list_skus:
            df_micro_level = df_sales_dept_id_store_id[df_sales_dept_id_store_id.item_id == current_sku].copy()

            # Create a DF as a placeholder and fill it with data
            df_temp = pd.DataFrame({'date':idx_dates, 'store_nbr': current_store_id, 'item_id': current_sku})
            # let's do a left join
            df_merged = pd.merge(df_temp, df_micro_level, how='left', on=['date', 'store_nbr', 'item_id'],copy=True)
            df_merged.sales.fillna(0.0, inplace=True)
            df_merged.onpromotion.fillna(False, inplace=True)
            df_merged.sort_values(by=['date'], ascending=True)

            # update the main DF
            sales_var = f'sales-{dept_id}_{current_sku}-{store_name}'
            promo_var = f'promotion_flag-{dept_id}_{current_sku}-{store_name}'
            df_store_sales[sales_var] = df_merged['sales']
            df_store_sales[promo_var] = df_merged['onpromotion']
            total_units += df_merged['sales']

        # update the total sales
        df_store_sales['total_units']=total_units

        # save the info
        foldername = os.path.join(os.path.expanduser(outputBaseFolder), cat_id, dept_id, 'store_sales')
        fhelp.makeFolder(foldername)
        filename = os.path.join(foldername, f'{store_name}.pickle')

        fhelp.toPickleFile(df_store_sales, filename)
        print(f'Saving {filename}...')

0/54-Working on 25...
Number of SKUs 20
Saving /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/data/CorporacionFavorita/AUTOMOTIVE/AUTOMOTIVE/store_sales/Santa_Elena_25_D_1.pickle...
1/54-Working on 1...
Number of SKUs 20
Saving /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/data/CorporacionFavorita/AUTOMOTIVE/AUTOMOTIVE/store_sales/Pichincha_1_D_13.pickle...
2/54-Working on 2...
Number of SKUs 20
Saving /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/data/CorporacionFavorita/AUTOMOTIVE/AUTOMOTIVE/store_sales/Pichincha_2_D_13.pickle...
3/54-Working on 3...
Number of SKUs 20
Saving /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/data/CorporacionFavorita/AUTOMOTIVE/AUTOMOTIVE/store_sales/Pichincha_3_D_8.pickle...
4/54-Working on 4...
Number of SKUs 20
Saving /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/data/CorporacionFavorita/AUTOMOTIVE/AUTOMOTIVE/store_sales/Pichincha_4_D_9.pickle...
5/54-Working on 5...
Num

### Let's come up with the store sales per dept_id

In [ ]:
dept_id = 'MEATS' # comes from dept_ids
idx_dates = pd.date_range('2017-01-01', '2017-08-15')
cat_id = mapper_dept_to_category[dept_id]

In [ ]:
# Find the sales for the current dept_id
idx_dept_id = df_items.dept_id==dept_id
list_skus = df_items[idx_dept_id].item_id.unique().tolist()

df_sales_dept_id = df_train[df_train.item_id.isin(list_skus)].copy()
df_sales_dept_id.head(2)

In [ ]:
list_store_id = df_sales_dept_id.store_nbr.unique().tolist()
# I'll loop through the stores
current_store_id = list_store_id[0]
current_store_id = 44
df_sales_dept_id_store_id = df_sales_dept_id[df_sales_dept_id.store_nbr==current_store_id]
df_sales_dept_id_store_id.head(2)

In [ ]:
list_store_id = df_sales_dept_id.store_nbr.unique().tolist()
total_stores = len(list_store_id)

for idx_this_store, current_store_id in enumerate(list_store_id):

    print(f'{idx_this_store}/{total_stores}-Working on {current_store_id}...')
    
    df_sales_dept_id_store_id = df_sales_dept_id[df_sales_dept_id.store_nbr==current_store_id]
    total_units = np.zeros(shape=idx_dates.shape[0])
    df_store_sales = pd.DataFrame({'date':idx_dates, 'weekdays':idx_dates.day_name()})
    
    store_name = mapper_store_id[current_store_id]
    num_skus = len(list_skus)

    print(f'Number of SKUs {num_skus}')
    
    for current_sku in list_skus:
        df_micro_level = df_sales_dept_id_store_id[df_sales_dept_id_store_id.item_id == current_sku].copy()

        # Create a DF as a placeholder and fill it with data
        df_temp = pd.DataFrame({'date':idx_dates, 'store_nbr': current_store_id, 'item_id': current_sku})
        # let's do a left join
        df_merged = pd.merge(df_temp, df_micro_level, how='left', on=['date', 'store_nbr', 'item_id'],copy=True)
        df_merged.sales.fillna(0.0, inplace=True)
        df_merged.onpromotion.fillna(False, inplace=True)
        df_merged.sort_values(by=['date'], ascending=True)

        # update the main DF
        sales_var = f'sales-{dept_id}_{current_sku}-{store_name}'
        promo_var = f'promotion_flag-{dept_id}_{current_sku}-{store_name}'
        df_store_sales[sales_var] = df_merged['sales']
        df_store_sales[promo_var] = df_merged['onpromotion']
        total_units += df_merged['sales']

    # update the total sales
    df_store_sales['total_units']=total_units
    
    # save the info
    foldername = os.path.join(os.path.expanduser(outputBaseFolder), cat_id, dept_id, 'store_sales')
    fhelp.makeFolder(foldername)
    filename = os.path.join(foldername, f'{store_name}.pickle')

    fhelp.toPickleFile(df_store_sales, filename)
    print(f'Saving {filename}...')

In [ ]:
store_name = 'Pichincha_47'
filename = os.path.join(foldername, f'{store_name}.pickle')
df_A = fhelp.readPickleFile(filename)
df_A.head(3)



promo_vars = [iVar for iVar in df_A.columns.tolist() if 'promotion_flag-' in iVar]
sku_promos = df_A[promo_vars].sum(axis=0)

df_A['simultaneous_promos'] = df_A[promo_vars].sum(axis=1)

idx_promos = df_A['simultaneous_promos'] > 10

x_axis, y_data = df_A.date, df_A.total_units

def_colours = plt.rcParams['axes.prop_cycle'].by_key()['color']
# Plot one store
f, ax = plt.subplots(1,1,figsize=(fig_w*1.5, fig_h/1.5))
ax.plot(x_axis, y_data, color=def_colours[3], linewidth=2, alpha=0.75)

ax.plot(x_axis[idx_promos], y_data[idx_promos], '*', color=def_colours[4], linewidth=2, alpha=0.75)
#plt.legend()
plt.xlabel('dates')
plt.grid(True)
plt.show()


In [ ]:
sku_A = 'sales-MEATS_582864-Pichincha_47'
fhelp.simple_XY_plot(df_A.date, df_A[sku_A])

In [ ]:
df_LOESS = fhelp.decompose_signal(df_A[sku_A], period_in_days=14, minimum_heartbeat=0.85)
df_LOESS['date'] = df_A.date
df_LOESS[sku_A] = df_A[sku_A]
df_LOESS.head()

In [ ]:
df_A.head()

promo_vars = [iVar for iVar in df_A.columns.tolist() if 'promotion_flag-' in iVar]
sku_promos = df_A[promo_vars].sum(axis=0)

simultaneous_promos = df_A[promo_vars].sum(axis=1)

simultaneous_promos[0:1]

In [ ]:
promo_vars[0:2]

In [ ]:
fhelp.plot_LOESS_decomposition(df_LOESS, sku_A)